In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

from scipy.spatial import distance
import numpy as np

from KeyEval import *
from tqdm import tqdm

## Textrank (no docs)

In [9]:
model = PKE("singlerank")

In [10]:
all_scores, all_scores_adjust = eval_file("Datasets/DataFiles/bib_tug_dataset_full.parquet", model,model_param = [""] ,year1=2019, year2=2019, types=["compsci"] , log=True)
print(all_scores, all_scores_adjust )


  0%|                                                                                           | 0/33 [00:00<?, ?it/s]

(33, 22)
2019    33
Name: year, dtype: int64


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:43<00:00,  1.33s/it]

[0.013756163622473784, 0.010514033616800415, 0.02121212121212121, 0.17224910293059145] [0.022815776052984127, 0.02463959651459651, 0.021875, 0.19686227545602536]


In [12]:
all_scores, all_scores_adjust = eval_file("Datasets/DataFiles/bib_tug_dataset_full.parquet", Textacy("scake"),model_param = [""] ,year1=2019, year2=2019, types=["compsci"] , log=True)
print(all_scores, all_scores_adjust )

  3%|██▌                                                                                | 1/33 [00:00<00:05,  6.10it/s]

(33, 22)
2019    33
Name: year, dtype: int64


100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:04<00:00,  6.91it/s]

[0.08293932691552149, 0.06150149364240514, 0.13333333333333333, 0.32405777877451225] [0.13848805603590483, 0.15122949966699964, 0.1375, 0.5371264730639731]


## CollabRank using clustering (kmeans)

In [2]:
keywords_gold, abstracts = get_key_abs("Datasets/DataFiles/bib_tug_dataset_full.parquet",count = 250, rand =  True)

(250, 22)
2009    19
1992    16
1993    16
2010    16
2008    14
1991    13
1994    12
1995    10
2007     9
1990     9
1978     8
2006     7
1975     7
1985     7
1989     7
2005     6
1988     6
1983     5
1986     5
1980     4
1999     4
1976     3
1984     3
1997     3
2013     3
2011     3
1996     3
1973     3
2003     3
2004     3
2015     2
2012     2
1987     2
1971     2
1981     2
1974     1
1977     1
1972     1
1970     1
2018     1
1979     1
1982     1
2016     1
2000     1
2001     1
2002     1
2014     1
1969     1
Name: year, dtype: int64


In [3]:
docs =abstracts

In [4]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(docs)

In [5]:
k = 5

In [6]:
model = KMeans(n_clusters=k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

KMeans(max_iter=100, n_clusters=5, n_init=1)

In [14]:
model.labels_

array([0, 0, 0, 1, 0, 1, 2, 4, 1, 4, 0, 3, 2, 0, 2, 1, 3, 2, 0, 4, 0, 4,
       2, 0, 3, 3, 4, 3, 3, 3, 1, 4, 0, 2, 4, 2, 1, 3, 4, 4, 4, 1, 2, 2,
       3, 0, 0, 4, 1, 0, 0, 3, 0, 0, 3, 3, 0, 0, 2, 0, 1, 3, 4, 2, 4, 1,
       4, 2, 4, 2, 1, 2, 3, 2, 1, 4, 1, 3, 2, 4, 4, 0, 1, 0, 4, 0, 0, 4,
       2, 2, 0, 2, 4, 2, 1, 2, 3, 4, 3, 0, 2, 4, 2, 0, 4, 2, 2, 4, 4, 2,
       3, 2, 2, 0, 0, 4, 0, 3, 4, 0, 3, 0, 3, 2, 4, 2, 0, 3, 1, 3, 4, 0,
       4, 4, 2, 0, 4, 0, 4, 3, 1, 1, 4, 0, 3, 4, 2, 4, 4, 2, 2, 4, 1, 2,
       0, 2, 0, 3, 1, 3, 3, 2, 3, 4, 0, 4, 4, 1, 3, 3, 4, 1, 3, 0, 0, 2,
       2, 2, 0, 2, 2, 2, 1, 2, 2, 4, 2, 4, 1, 0, 2, 1, 4, 2, 4, 4, 4, 3,
       3, 2, 4, 3, 2, 1, 4, 2, 0, 0, 2, 2, 1, 1, 0, 2, 2, 2, 2, 0, 0, 4,
       2, 1, 3, 1, 2, 0, 4, 4, 0, 1, 0, 1, 0, 0, 3, 2, 4, 4, 4, 1, 0, 2,
       2, 4, 3, 3, 2, 4, 2, 2])

In [15]:
keywords_gold, abstracts = get_key_abs("Datasets/DataFiles/bib_tug_dataset_full.parquet",year1 = 2019, year2 = 2019, types=["compsci"])

(33, 22)
2019    33
Name: year, dtype: int64


In [16]:
new_documents = abstracts

In [17]:
collab_documents = []
for d in new_documents:
    d = new_documents[0]
    X = vectorizer.transform([d])
    min_d = (100,0)
    for center in range(k):
        dist = distance.cosine(X[0].toarray(), np.array(model.cluster_centers_[center]))
        if dist < min_d[0]:
            min_d=(dist, center)
    collab_docs = []
    for i in range(len(model.labels_)):
        label = model.labels_[i]
        if label == min_d[1]:
            collab_docs.append((docs[i],min_d[0]))
    collab_documents.append(collab_docs)


In [18]:
all_scores = []
all_scores_adjust = []
T0 = 0.0
T1 = 0.0

In [19]:
for i in tqdm(range(len(keywords_gold))):

    t3 = time.time()
    model = CollabRank(collab_documents[i])
    predicted_keywords = model.get_keywords(abstracts[i])
    #open("C:/Users/dallal/Documents/GitHub/keyphrase/temp/"+str(i)+".txt", "w").write(abstracts[i])

    t4 = time.time()
    try:
        scores = get_all_scores(keywords_gold[i], predicted_keywords[0], abstracts[i], adjust=True)

    except:
        scores = [[0.0, 0.0, 0.0,0.0] , [0.0, 0.0, 0.0,0.0]]

    t5 = time.time()
    all_scores.append(scores[0])
    if scores[1]:
        all_scores_adjust.append(scores[1])
    T0 += (t4 - t3)
    T1 += (t5 - t4)
all_scores = pd.DataFrame(all_scores)
all_scores_adjust = pd.DataFrame(all_scores_adjust)

  0%|                                                                                           | 0/33 [00:00<?, ?it/s]

a new performance model for dynamic locking is proposed. it is based on a flow diagram and uses only the steady state average values of the variables. it is general enough to handle nonuniform access, shared locks, static locking, multiple transaction classes, and transactions of indeterminate length. the analysis is restricted to the case in which all conflicts are resolved by restarts. the model shows that data contention can cause the throughput to thrash, and gives a limit on the workload that will prevent this. it also shows that systems with a particular kind of nonuniform access and systems in which transactions share locks are equivalent to systems in which there is uniform access and only exclusive locking. replacing updates by queries in a multiprogramming mix may degrade performance if the queries are longer than the updates.
describes three ada packages which support the teaching of some recent and important ideas in numerical computation. the first is designed to enable st

ml modules provide hierarchical namespace management, as well as fine-grained control over the propagation of type information, but they do not allow modules to be broken up into mutually recursive, separately compilable components. mixin modules facilitate recursive linking of separately compiled components, but they are not hierarchically composable and typically do not support type abstraction. we synthesize the complementary advantages of these two mechanisms in a novel module system design we call mixml.\par a mixml module is like an ml structure in which some of the components are specified but not defined. in other words, it unifies the ml structure and signature languages into one. mixml seamlessly integrates hierarchical composition, translucent mlstyle data abstraction, and mixin-style recursive linking. moreover, the design of mixml is clean and minimalist; it emphasizes how all the salient, semantically interesting features of the ml module system (as well as several propos

the paper provides the rationale for the architecture and design of a color image display and processing system called hacienda. the system was heavily influenced by one of its most important intended applications, the processing of landsat data, including that to be provided by landsat d. also considered in the paper are the trade-offs involved in making the system suitable for a broader range of image processing work without unduly adding to cost or complexity.
many of the ada 9x mappings will have a dramatic impact on current implementations of ada run-time executives. this impact is particularly severe for the case of run-time systems that are implemented with protected kernels, often required in real-time military embedded systems. further, we believe that run-time systems that provide these new 9x facilities will be required to use techniques that are less efficient than current run-time systems at providing ada 83 tasking services. consequently, existing ada systems that meet pe

the utilization of storage is studied in a two-level memory hierarchy. the first storage level, which is the fast store, is divided into a number of storage areas. when an entry is to be filed in the hierarchy, a hashing algorithm will attempt to place the entry into one of these areas. if this particular area is full, then the entry will be placed into the slower second-level store, even though other areas in the first-level store may have space available. given that n entries have been filed in the entire hierarchy, an expression is derived for the expected number of entries filed in the first-level store. this expression gives a measure of how effectively the first-level store is being used. by means of examples, storage utilization is then studied as a function of the hashing algorithm, the number of storage areas into which the first-level store is divided and the total size of the first-level store.
java is a programming language loosely related to c++. java originated in a proje

...

the paper provides the rationale for the architecture and design of a color image display and processing system called hacienda. the system was heavily influenced by one of its most important intended applications, the processing of landsat data, including that to be provided by landsat d. also considered in the paper are the trade-offs involved in making the system suitable for a broader range of image processing work without unduly adding to cost or complexity.
many of the ada 9x mappings will have a dramatic impact on current implementations of ada run-time executives. this impact is particularly severe for the case of run-time systems that are implemented with protected kernels, often required in real-time military embedded systems. further, we believe that run-time systems that provide these new 9x facilities will be required to use techniques that are less efficient than current run-time systems at providing ada 83 tasking services. consequently, existing ada systems that meet pe

the utilization of storage is studied in a two-level memory hierarchy. the first storage level, which is the fast store, is divided into a number of storage areas. when an entry is to be filed in the hierarchy, a hashing algorithm will attempt to place the entry into one of these areas. if this particular area is full, then the entry will be placed into the slower second-level store, even though other areas in the first-level store may have space available. given that n entries have been filed in the entire hierarchy, an expression is derived for the expected number of entries filed in the first-level store. this expression gives a measure of how effectively the first-level store is being used. by means of examples, storage utilization is then studied as a function of the hashing algorithm, the number of storage areas into which the first-level store is divided and the total size of the first-level store.
java is a programming language loosely related to c++. java originated in a proje

we present a method of automatic generation of functional vectors for sequential circuits. these vectors can be used for design verification, manufacturing testing, or power estimation. a high-level description of the circuit in vhdl or c is assumed available. our method automatically transforms the high-level description of a circuit in vhdl or c into an extended finite state machine (efsm) model that is used to generate functional vectors. the efsm model is a generalization of the traditional state machine model. it is a compact representation of models with local data variables and preserves many nice properties of a traditional state machine model. the theoretical background of the efsm model is addressed in this article. our method guarantees that the generated vectors cover every statement in the high-level description at least once. experimental results show that a set of comprehensive functional vectors for sequential circuits with more than a hundred flip-flops can be generate

a methodology for benchmarking dedicated, interactive systems has been developed at the mitre corporation. this methodology uses a synthetic program model of the application which runs on the proposed hardware/operating system configurations and is driven by a statistically derived load. system performance is measured by analyzing the synthetic transaction response times. the methodology yields assurances to a buyer that the benchmarked system has at least an a priori defined amount of computer power available for applications-oriented software. this paper examines the methodology and the problems that were encountered and solutions which have been used in calibrating a benchmark model for a specific application. the benchmark was designed to model a large interactive information processing application on a procurement requiring loosely-coupled (no shared memory) multicomputer systems. the model consists of a set of interacting synthetic program cells, each composed of several abstract

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [48:05<00:00, 87.43s/it]


In [20]:
all_scores = list(all_scores.mean(axis=0))


In [21]:
all_scores

[0.09443882496953493,
 0.06921998161957657,
 0.15454545454545457,
 0.28055005354691265]

## Test Bib-CollabRank

In [100]:

keywords_gold_bib, abstracts_bib = get_key_abs("Datasets/DataFiles/bib_tug_dataset_full.parquet",year1 = 2017, year2 = 2018, types=["compsci"])

(54, 22)
2017    32
2018    22
Name: year, dtype: int64


In [102]:
collab_documents_bib=[]
for i in range(len(abstracts_bib)):
    collab_documents_bib.append((abstracts_bib[i],1.0))

In [103]:
model = CollabRank(collab_documents_bib)

In [104]:
all_scores, all_scores_adjust = eval_file("Datasets/DataFiles/bib_tug_dataset_full.parquet", model,model_param = ["weights20172018"] ,year1=2019, year2=2019, types=["compsci"] ,bib_weights={}, log=True)
print(all_scores, all_scores_adjust )


  0%|                                                                                           | 0/33 [00:00<?, ?it/s]

(33, 22)
2019    33
Name: year, dtype: int64
in deep sub-micron region, spin transfer torque ram (stt-ram) shows read-disturbance error (rde) which presents a crucial reliability challenge. we present shield, a technique to mitigate rde in stt-ram last level caches (llcs). shield uses data compression to reduce cache-write traffic and restore requirement. also, shield keeps two copies of data blocks compressed to less than half the block size and since several llc blocks are only accessed once, this approach avoids several restore operations. shield consumes smaller energy than two previous rde-mitigation techniques, namely high-current restore required read (hcrr, also called restore-after-read) and low-current long latency read (lcll) and even an ideal rde-free stt-ram cache.
current memory technology has hit a wall trying to scale to meet the increasing demands of modern client and datacenter systems. data compression is a promising solution to this problem. several compressed memor

approximate matrix inversion based on neumann series has seen a recent increased interest motivated by massive mimo systems. there, the matrices are in many cases diagonally dominant, and, hence, a reasonable approximation can be obtained within a few iterations of a neumann series. in this work, we clarify that the complexity of exact methods are about the same as when three terms are used for the neumann series, so in this case, the complexity is not lower as often claimed. the second common argument for neumann series approximation, higher parallelism, is indeed correct. however, in most current practical use cases, such a high degree of parallelism is not required to obtain a low latency realization. hence, we conclude that a careful evaluation, based on accuracy and latency requirements must be performed and that exact matrix inversion is in fact viable in many more cases than the current literature claims.
lifting-based complex multiplications and rotations are integer invertible

multiplication of large integers is a fundamental operation for public key cryptography. in contemporary public key cryptography, the sizes of integers are typically from more than one hundred bits to even several thousands of bits. because these sizes exceed the bit widths of all general-purpose processors, these multiplications must be performed with a multiprecision multiplication algorithm which splits the operation into multiple partial products and accumulation steps. to ensure efficiency, multiprecision multiplication algorithms must be designed with special care and optimized for the instruction sets of specific processors. consequently, developing efficient multiprecision multiplication algorithms and optimizing them for specific platforms has been an active research topic. in this paper, we optimize multiprecision multiplication and squaring specifically for the 64-bit armv8 processors which are widely used, for example, in modern smart phones and tablets. we combine the subt

users of financial and e-commerce services demand a high degree of reliability and at the same time an increasing demand of speed of processing. on the other hand, soft errors are becoming more significant due to the higher densities and reduced cmos integration technology sizes. among the basic arithmetic operations, addition/subtraction is the most demanded. although in the past, binary implementations were only considered, today decimal implementations are becoming important. in this context, we introduce a modular design for fast error checking of binary and decimal (bcd) addition/subtraction operations that avoids the whole replication of the arithmetic units. unlike other error checkers based on parity prediction or residue checking, this is a separable design that lies completely off of the critical path of the protected adder without incurring in important penalties in area or performance.
reliable implementation of digital filters in finite precision is based on accurate error

...

users of financial and e-commerce services demand a high degree of reliability and at the same time an increasing demand of speed of processing. on the other hand, soft errors are becoming more significant due to the higher densities and reduced cmos integration technology sizes. among the basic arithmetic operations, addition/subtraction is the most demanded. although in the past, binary implementations were only considered, today decimal implementations are becoming important. in this context, we introduce a modular design for fast error checking of binary and decimal (bcd) addition/subtraction operations that avoids the whole replication of the arithmetic units. unlike other error checkers based on parity prediction or residue checking, this is a separable design that lies completely off of the critical path of the protected adder without incurring in important penalties in area or performance.
reliable implementation of digital filters in finite precision is based on accurate error

quotient selection (qs) is a key step in the classic $ o(n^2) $ multiple precision division algorithm. on processors with fast hardware division, it is a trivial problem, but on gpus, division is quite slow. in this paper we investigate the effectiveness of brent and zimmermann's variant as well as our own novel variant of barrett's algorithm. our new approach is shown to be suitable for low radix (single precision) qs. three highly optimized implementations, two of the brent and zimmerman variant and one based on our new approach, have been developed and we show that each is many times faster than using the division operation built in to the compiler. in addition, our variant is on average 22\% faster than the other two implementations. we also sketch proofs of correctness for all of the implementations and our new algorithm.
the ieee 754-2008 standard governs floating-point arithmetic in all types of computer systems. the standard provides for two radices, 2 and 10. it specifies conv

deep learning has been undergoing rapid growth in recent years thanks to its state-of-the-art performance across a wide range of real-world applications. traditionally neural networks were trained in ieee-754 binary64 or binary32 format, a common practice in general scientific computing. however, the unique computational requirements of deep neural network training workloads allow for much more efficient and inexpensive alternatives, unleashing a new wave of numerical innovations powering specialized computing hardware. we previously presented flexpoint, a blocked fixed-point data type combined with a novel predictive exponent management algorithm designed to support training of deep networks without modifications, aiming at a seamless replacement of the binary32 widely in practice today. we showed that flexpoint with 16-bit mantissa and 5-bit shared exponent (flex16+s) achieved numerical parity to binary32 in training a number of convolutional neural networks. in the current paper we 

100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [35:18<00:00, 64.19s/it]

[0.12735060150535676, 0.09347078227613226, 0.2090909090909091, 0.2990358498295214] [0.21596661895269967, 0.23684752747252744, 0.215625, 0.4930468851171977]


In [105]:
print(all_scores, all_scores_adjust )

[0.12735060150535676, 0.09347078227613226, 0.2090909090909091, 0.2990358498295214] [0.21596661895269967, 0.23684752747252744, 0.215625, 0.4930468851171977]
